# Caracterization of 2-qubit states

We are hoping to caracterize the families of states of the format
$$\rho_{AB}^{\eta} = \eta\rho_{AB}+(1-\eta)\mathbb{I}.$$

I will be creating the target state ($\rho_{AB}$) either by the Bures metric or the Hilbert-Schmidt metric (this part is already finalized).

The tests we are planning on doing is:
- Entanglement
    - PPT test (finding the $\eta_{\text{ent}}$, for which the family begins to be entangled)
        - This part is already finalized.
- Bell's non-locality 
    - Lower bound: SDP
    - Upper bound: Bell's inequalities
- Steering 
    - Upper bound: Sew Saw
    - Lower bound: SDP
        - This part is already finalized.

![Caracterization of the states](caracterization.png)

In [8]:
# Importing the necessary libraries

import numpy as np
from numpy import linalg as LA
from random import random
from scipy.stats import unitary_group
import picos as pic

In [9]:
# Functions to create the target state

# Generation of the random matrix from the Ginibre ensemble
def G_matrix(m,n):
    # Matrix G of size m x n
    G = np.zeros((m,n),dtype=np.complex_)
    for k in range(m):
        for l in range(n):
            G[k,l] = random()+random()*1j
    return G

# Generation a random mixed density matrix (Bures metric)
def rho_mixed(n):
    # n = dimension of the state    
    # Create random unitary matrix
    U = unitary_group.rvs(n)
    # Create random Ginibre matrix
    G = G_matrix(n,n)
    # Create identity matrix
    I = np.eye(4)
    # Construct density matrix
    rho = (I+U)*G*(G.conjugate().T)*(I+U.conjugate().T)
    # Normalize density matrix
    rho = rho/(rho.trace())
    return rho

# Generation a random mixed density matrix (Hilbert-Schmidt metric)
def rho_mixed_HS(n):
    # n = dimension of the state  
    # Create random Ginibre matrix
    G = G_matrix(n,n)
    # Construct density matrix
    rho = G*(G.conjugate().T)
    # Normalize density matrix
    rho = rho/(rho.trace())
    return rho

In [10]:
# Generating the target state (Werner state)

#Create the base kets |00>, |01>, |10> and |11>
ket_00 = np.array([[1],[0],[0],[0]])
ket_01 = np.array([[0],[1],[0],[0]])
ket_10 = np.array([[0],[0],[1],[0]])
ket_11 = np.array([[0],[0],[0],[1]])
#Create |psi> = (|01>-|10>)/sqrt(2) and rho = |psi><psi|
psi = (ket_01-ket_10)/np.sqrt(2)
rho_AB = psi@np.transpose(psi)

# Generating the separable state

rho_I = np.eye(4)/4

## Entanglement

The function `Ent_cert(rho)` will say if the state is entangled or not, by calculating the partial transpose of the state and checking if it is positive.

Here we find the $\eta_{\text{ent}}$ in a precision of 


In [11]:
# Defining the function that certifies entanglement

def Ent_cert(rho):
    # Calculate partial transpose
    n = rho.shape
    rho_TA = np.zeros((n[0],n[1]),dtype=np.complex_)
    a = int(n[0]/2)
    b = int(n[1]/2)
    rho_TA[:a,:b] = rho[:a,:b]
    rho_TA[a:,b:] = rho[a:,b:]
    rho_TA[a:,:b] = rho[a:,:b].T
    rho_TA[:a,b:] = rho[:a,b:].T
    # v - eigenvectors, w - eigenvalues
    w, v = LA.eig(rho_TA)
    # PPT Criterion: Are all eigenvalues >=0?
    if all(i >= 0 for i in w):
        # print('Yes: separable state.')
        ppt = 0
    else:
        # print('No: entangled state.')
        ppt = 1
    return w,v,ppt

In [12]:
# Calculating the limit of entanglement

r = 0.5
eps = 0.5
ppa = 0

i = 0

while eps >= 10**(-15) and r>=0 and r<=1 and i<=10**5:
    rho_ent = r*rho_AB+(1-r)*rho_I
    w,v, ppt = Ent_cert(rho_ent)

    eps = eps/2

    if ppt == 0:
        r = r + eps
    else:
        r = r - eps

    i += 1

eta_ent = r

print(eta_ent)

0.33333333333333304


In [13]:
def ent_threshold(rho, rho_sep):
    '''
    Calculating the limit of entanglement
    Input: rho - target state (Array)
           rho_sep - separable state (Array)
    '''

    r = 0.5
    eps = 0.5
    i = 0

    while eps >= 10**(-15) and r>=0 and r<=1 and i<=10**5:
        rho_ent = r*rho_AB+(1-r)*rho_I
        w,v, ppt = Ent_cert(rho_ent)

        eps = eps/2

        if ppt == 0:
            r = r + eps
        else:
            r = r - eps

        i += 1

    eta_ent = r

    return eta_ent

# Generating the target state (Werner state)

#Create the base kets |00>, |01>, |10> and |11>
ket_00 = np.array([[1],[0],[0],[0]])
ket_01 = np.array([[0],[1],[0],[0]])
ket_10 = np.array([[0],[0],[1],[0]])
ket_11 = np.array([[0],[0],[0],[1]])
#Create |psi> = (|01>-|10>)/sqrt(2) and rho = |psi><psi|
psi = (ket_01-ket_10)/np.sqrt(2)
rho_AB = psi@np.transpose(psi)

# Generating the separable state

rho_I = np.eye(4)/4

print(ent_threshold(rho_AB,rho_I))

0.33333333333333304


In [14]:
# Implementação via SDP Hierarquia DPS

# Given 
#Create the base kets |00>, |01>, |10> and |11>
ket_01 = np.array([[0],[1],[0],[0]])
ket_10 = np.array([[0],[0],[1],[0]])

#Create |psi> = (|01>-|10>)/sqrt(2) and rho = |psi><psi|
psi = (ket_01-ket_10)/np.sqrt(2)

rho_AB = psi@np.transpose(psi)

k = 2

dimension = 2*2**(k)

#Creating the problem
P = pic.Problem()

#Creating the optimization variables

rho_k = pic.HermitianVariable('rho_k',(dimension,dimension))

#Creating the constraints
P.add_list_of_constraints([pic.partial_trace(rho_k, subsystems = i+1, dimensions = 2)==rho_AB for i in range(k)])
P.add_constraint(rho_k>>0)
P.add_constraint(pic.partial_transpose(rho_k, subsystems = 0, dimensions = 2)>>0)
P.add_constraint(pic.partial_transpose(rho_k, subsystems = 1, dimensions = 2)>>0)

#Setting the objective
P.set_objective('find',rho_k)

#Finding the solution
P.solve()

print(q)

## Bell's non-locality 

Bell scenarios are determined by the following finite values:
- the number of parts;
- the number of possible measurements of each part;
- the number of possible outcomes of each possible measurement of each
part.

### Lower bound: SDP or Outer polytope approximation

### Upper bound: Bell's inequalities - SewSaw

CHSH
$$\langle A_0 B_0\rangle+\langle A_1 B_0\rangle+\langle A_0 B_1\rangle−\langle A_1 B_1\rangle\leq 2$$

I3322
$$−\langle A_0\rangle−\langle A_1\rangle−\langle B_0\rangle−\langle B_1\rangle−\langle A_0 B_0\rangle−\langle A_1 B_0\rangle−\langle A_2 B_0\rangle−\langle A_0 B_1\rangle−\langle A_1 B_1\rangle−\langle A_2 B_1\rangle−\langle A_0 B_2\rangle+\langle A_1 B_2\rangle\leq 4$$

## Desigualdades de Bell
Seja $\vec{p}\in\mathcal{P}_Q$, de forma que 
$$p(ab|xy) = \text{tr}[(E_{a|x}\otimes F_{b|y})\rho],$$
onde $\rho$ é o estado do sistema, e $\{ E_{a|x}\}$ e $\{ F_{b|y}\}$ são os POVMs de Alice e Bob, respectivamente.

Neste contexto, uma desigualdade de Bell pode ser escrita como
$$D(\vec{p}) = c^T\vec{p}=\text{tr}[G\rho]\leq d_L$$
onde
$$G = \sum_{a,b,x,y}c(a,b,x,y)(E_{a|x}\otimes F_{b|y})$$
é um observável associado à desigualdade.

## Algoritmo See-Saw
O seguinte algoritmo iterativo converge para uma cota inferior da máxima violação quântica de uma desigualdade de Bell:
1. Dadas dimensões locais $d_A$ e $d_B$, sorteie POVMs $\{ E_{a|x}\}$ atuando em $\mathbb{C}^{d_A}$ e $\{ F_{b|y}\}$ atuando em $\mathbb{C}^{d_B}$;
2. Construa o observável $G$ associado à desigualdade, obtenha o autovetor $|\psi\rangle$ associado ao maior autovalor, e faça $\rho = |\psi\rangle\langle\psi|$;
3. Maximize $D(\rho, \{ E_{a|x}\}, \{ F_{b|y}\})$ sobre $\{ E_{a|x}\}$, mantendo $\rho$ e $\{ F_{b|y}\}$ fixos;
4. Maximize $D(\rho, \{ E_{a|x}\}, \{ F_{b|y}\})$ sobre $\{ F_{b|y}\}$, mantendo $\rho$ e $\{ E_{a|x}\}$ fixos;
5. Volte ao passo 2. e repita até convergência.
O algoritmo see-saw converge para um máximo local de $D(\rho, \{ E_{a|x}\}, \{ F_{b|y}\})$; é recomendável, portanto, tomar vários pontos iniciais distintos.

## Fixando o cenário
Iremos incrementar a dimensão ao longo do projeto, entretanto teremos $d_A = d_B = d$. As medições precisam respeitar as propriedades de POVM
$$M_{a|x}\geq0,\quad\sum_a M_{a|x}=\mathbb{I} \text{ e}\quad M_{a|x}^\dagger = M_{a|x}.$$
Sabemos que uma matrix mais a sua matrix hermitiana é uma matrix hermitiana. Assim, pra garantir a última propriedade, podemos criar uma matrix aleatória complexa $A$ e calcular a nossa medição $M_{a|x} = A + A.\texttt{conj}().\texttt{T}$.

Para garantir a segunda propriedade, podemos fixar o nosso cenário para medições dicotômicas, assim, encontrado $M_{a|x}$, fazemos $M_{a'|x} = \mathbb{I} - M_{a|x}$.
No caso de $k$ resultados por medição, criamos $k-1$ medições através da técnica de hermitianidade e para a última fazemos $M_{k|x} = \mathbb{I} - \sum^{k-1}_{i=1} M_{i|x}$.

Para garantir a primeira propriedade, devemos calcular o traço da matriz e ele deve ser não-negativo $\text{tr}[M_{a|x}]\geq0$.

In [15]:
def Haar_random(dimension):
    '''
    Generating random unitary matrix according to Haar measure.
    Ref.: https://arxiv.org/abs/math-ph/0609050v2

    Input: dimension (Integer)
    Output: array of the matrix
    '''
    A = G_matrix(dimension,dimension)
    q, r = np.linalg.qr(A)
    m = np.diagonal(r)
    m = m / np.abs(m)
    return np.multiply(q, m, q)


def proj_meas_random(dimension):
    '''
    Generating random projective measurement according to Haar measure.
    Input: dimension (integer)
    Output: array of the measurement
    '''

    Haar = Haar_random(dimension)

    measurement = np.zeros((dimension,dimension,dimension),dtype=complex)

    for i in range(dimension):
        M = np.zeros((dimension, dimension),dtype=complex)
        M[i][i] = 1
        measurement[i] = Haar@M@Haar.conj().T

    return measurement


def is_measurement(measurement):
    '''
    Checks if the array is a measurement.
    Input: array of measurement.
    Output: True/False
    '''

    k = measurement.shape[0]
    dim = measurement.shape[1]
    soma = np.zeros((dim, dim),dtype=complex)
    for i in range(k):
        soma = measurement[i]+soma
        w, v = np.linalg.eig(measurement[i])
        #Has to be positive semi-definite
        if not (np.all(w>=0)): return False
    
    #Has to sum identity
    is_it = np.allclose(soma, np.eye(dim,dtype=complex), rtol=1.e-7, atol=1.e-8)

    return is_it


# Haar = Haar_random(2)
# teste_1 = Haar@np.array([[1,0],[0,0]])@Haar.conj().T
# teste_2 = Haar@np.array([[0,0],[0,1]])@Haar.conj().T

# print(Haar)

# print(teste_1+teste_2)
# meas = proj_meas_random(4)
# print(meas)
# print(meas[0]+meas[1])

# print(np.allclose(meas[0]+meas[1], np.eye(2,dtype=complex), rtol=1.e-15, atol=1.e-15))


meas = np.array([[[0,0],[0,2]],[[1,0],[0,-1]]])
# w, v = np.linalg.eig(meas[1])
# print(w)
# w, v = np.linalg.eig(meas[0])
# print(w)
print(is_measurement(meas))

False


In [26]:
# Criando as medições
def measurement(dimension,k):

    measurement = np.zeros((k,dimension,dimension),dtype=complex)
    soma = np.zeros((dimension,dimension))
    trace = np.zeros(k)

    valid = 0

    while valid != k:
        for i in range(k-1):
            A = np.random.rand(dimension,dimension) + (1j)*(np.random.rand(dimension,dimension))
            measurement[i] = A + A.conj().T
            soma = soma + measurement[i]
            if np.real(np.trace(measurement[i])) : trace[i] = True
            else: trace[i] = False
        measurement[k-1] = np.eye(dimension) - soma
        if np.real(np.trace(measurement[k-1])) : trace[k-1] = True
        else: trace[k-1] = False
        valid = np.sum(trace)
    return measurement
teste = measurement(4,2)

is_it = is_measurement(teste)

print(teste)
print(is_it)

[[[ 0.77919955+0.j          0.71114544-0.12318439j
    0.96732064-0.447207j    1.25721112+0.19332388j]
  [ 0.71114544+0.12318439j  0.65675104+0.j
    0.37016451-0.48265791j  0.36041902+0.31630418j]
  [ 0.96732064+0.447207j    0.37016451+0.48265791j
    0.62771212+0.j          0.19243339+0.46897658j]
  [ 1.25721112-0.19332388j  0.36041902-0.31630418j
    0.19243339-0.46897658j  1.37797057+0.j        ]]

 [[ 0.22080045+0.j         -0.71114544+0.12318439j
   -0.96732064+0.447207j   -1.25721112-0.19332388j]
  [-0.71114544-0.12318439j  0.34324896+0.j
   -0.37016451+0.48265791j -0.36041902-0.31630418j]
  [-0.96732064-0.447207j   -0.37016451-0.48265791j
    0.37228788+0.j         -0.19243339-0.46897658j]
  [-1.25721112+0.19332388j -0.36041902+0.31630418j
   -0.19243339+0.46897658j -0.37797057+0.j        ]]]
False


In [17]:
# def inequality_operator(alpha, rho, A0, A1, B0, B1):
#     """Expression must have picos variables, otherwise @ and * will get mixed up!"""
#     return rho * (alpha * A0 @ np.eye(2) + A0 @ B0 + A1 @ B0 + A0 @ B1 - A1 @ B1)

# def optimize_observables(alpha, rho, X0, X1, side, verb=0):
#     """Optimize the tilted CHSH over either `alice` or `bob` side."""
#     prob = pc.Problem()
#     X0, X1 = pc.Constant(X0), pc.Constant(X1)
#     O = [pc.HermitianVariable(f"O({i})", 2) for i in range(2)]
#     prob.add_list_of_constraints([o + eye(2) >> 0 for o in O])
#     prob.add_list_of_constraints([eye(2) - o >> 0 for o in O])
#     if side == "alice":
#         prob.set_objective("max", pc.trace(inequality_operator(alpha, rho, O[0], O[1], X0, X1)).real)
#     elif side == "bob":
#         prob.set_objective("max", pc.trace(inequality_operator(alpha, rho, X0, X1, O[0], O[1])).real)
#     return prob.solve(solver=SOLVER, verbose=verb), pc.Constant(O[0].value), pc.Constant(O[1].value)
    
# def see_saw(rho, rounds, inequality, dim_A):
#     """
#     See-saw algorithm for fixed density matrix rho and inequality
#     Input: rho - density matrix (array)
#            rounds - number of rounds to run (integer)
#            inequality - Bell's inequality (array) 
#            dim_A - Alice's dimension
#     """

#     for _ in range(rounds):

#         for 
#         prob, B0, B1 = optimize_observables(alpha, rho, A0, A1, side="bob")
#         prob, A0, A1 = optimize_observables(alpha, rho, B0, B1, side="alice")

#     return prob

Para obtermos o autovetor $|\psi\rangle$ associado ao maior autovalor, e recuperarmos $\rho = |\psi\rangle\langle\psi|$, precisamos resolver o problema de otimização:
$$\begin{align*}
\text{given} \quad & G \quad (c,\{E_{a|x}\},\{F_{b|y}\})\\
\max \quad &\text{tr}[G\rho] = \sum_{a,b,x,y}c(a,b,x,y)\text{tr}[(E_{a|x}\otimes F_{b|y})\rho]\\
\text{subject to}\quad & \text{tr}[\rho] = 1 \quad \text{Normalização} \\
\quad & \text{tr}[\rho^2] = 1 \quad \text{Estado puro}
\end{align*}
$$

In [18]:

#Isso não tá funcionando e não vai ser usado inicialmente, então vamos ignorar.
# Quando eu precisar otimizar pelo estado eu vou otimizar por assemblage de qualquer forma, então ignorar mesmo

# # Otimização do estado
# def max_rho(dimension, c, measurements_A, measurements_B):

#     #Creating the problem
#     P = pic.Problem()

#     #Creating the optimization variables

#     psi = pic.ComplexVariable('psi',(1,dimension))

#     #Creating the constraints
#     #P.add_constraint((psi.conj.T)@psi==1)
    
#     m_A = int(measurements_A.shape[0]/2)
#     m_B = int(measurements_B.shape[0]/2)

#     O_A = [measurements_A[2*i]-measurements_A[2*i+1] for i in range(m_A)]
#     O_B = [measurements_B[2*i]-measurements_B[2*i+1] for i in range(m_B)]

#     trace = [((psi.conj.T)@(np.kron(O_A[i],O_B[j]))@psi) for i in range(m_A) for j in range(m_B)]

#     D = pic.sum([c[i]*trace[i] for i in range(c.shape[0])])

#     #Setting the objective
#     P.set_objective('max',D)

#     #Finding the solution
#     P.solve()

#     rho = psi*(psi.conj.T)
#     return rho

# M_A = np.zeros((4,2,2))
# M_B = np.zeros((4,2,2))

# for i in range(2):
#     M_A[2*i], M_A[2*i+1] = measurement(2)
#     M_B[2*i], M_B[2*i+1] = measurement(2)

# print(M_A)
# c = G_CHSH()

# rho = max_rho(4,c,M_A,M_B)

In [19]:
def G_CHSH():
    '''
    CHSH inequality

    D = \sum_{a,b,x,y}c(a,b,x,y)\text{tr}[(E_{a|x} \otimes F_{b|y})\rho

    D = a_{+,0}xb_{+,0} - a_{+,0}xb_{-,0} - a_{-,0}xb_{+,0} + a_{-,0}xb_{-,0} + 
        a_{+,0}xb_{+,1} - a_{+,0}xb_{-,1} - a_{-,0}xb_{+,1} + a_{-,0}xb_{-,1} +
        a_{+,1}xb_{+,0} - a_{+,1}xb_{-,0} - a_{-,1}xb_{+,0} + a_{-,1}xb_{-,0} -
        a_{+,1}xb_{+,1} + a_{+,1}xb_{-,1} + a_{-,1}xb_{+,1} - a_{-,1}xb_{-,1}
    '''

    #a_{+,0}xb_{+,0} + a_{+,0}xb_{+,0} - a_{+,0}xb_{+,0} - a_{+,0}xb_{+,0} + 
    c = np.array([1,-1,-1,1,1,-1,-1,1,1,-1,-1,1,-1,1,1,-1])
    return c
c = G_CHSH()
print(c)

[ 1 -1 -1  1  1 -1 -1  1  1 -1 -1  1 -1  1  1 -1]


## Otimização sobre as medições de cada parte
Queremos maximizar $D(\rho, \{ E_{a|x}\}, \{ F_{b|y}\})$ sobre $\{ E_{a|x}\}$, mantendo $\rho$ e $\{ F_{b|y}\}$ fixos. Para isso vamos realizar o seguinte protocolo de otimização
$$\begin{align*}
\text{given} \quad & c,\rho,\{F_{b|y}\}\\
\max \quad & \sum_{a,b,x,y}c(a,b,x,y)\text{tr}[(E_{a|x}\otimes F_{b|y})\rho]\\
\text{subject to}\quad & E_{a|x}\geq0,\quad\forall a,x \\
\quad & \sum_a E_{a|x} = \mathbb{I}, \quad \forall x
\end{align*}
$$

In [20]:
# Otimização das medições
def max_Measurements_Alice(dimension,c,rho,measurements_B,k_A,m_A):

    '''
    Inputs:
    dimension - dimension of the measurements (Integer)
    c - Bell inequality constants (List of floats)
    rho - target state (Complex array)
    measurements_B - Bob's measurements (Complex array)
    k_A - number of results per Alices's measurement (Integer)
    m_A - number of Alice's measurements (Integer)

    Output:
    P - Problem created
    measurements_B - Bob's measurements created
    D - Value of Bell's inequality 
    solution - Result of the problem 
    '''

    k_B = int(measurements_B.shape[1])
    m_B = int(measurements_B.shape[0]/k_B)

    #Creating the problem
    P = pic.Problem()

    #Creating the optimization variables

    measurements_A = [pic.HermitianVariable('Measurements_Alice[{}]'.format(i),(dimension,dimension)) for i in range(k_A*m_A)]

    #Creating the constraints
    P.add_list_of_constraints([measurements_A[i]>>0 for i in range(k_A*m_A)])

    ident = [pic.sum([measurements_A[m_A*j+i] for i in range(k_A)]) for j in range(m_A)]

    P.add_list_of_constraints([ident[j]==np.eye(dimension,dtype='complex') for j in range(m_A)]) 

    trace = [pic.trace((measurements_A[i]@measurements_B[j])*rho) for i in range(m_A*k_A) for j in range(m_B*k_B)]

    D = pic.sum([c[i]*trace[i] for i in range(c.shape[0])])

    #Setting the objective
    P.set_objective('max',D)

    #Finding the solution
    solution = P.solve()

    return P, measurements_A, D, solution

def max_Measurements_Bob(dimension,c,rho,measurements_A,k_B,m_B):

    '''
    Inputs:
    dimension - dimension of the measurements (Integer)
    c - Bell inequality constants (List of floats)
    rho - target state (Complex array)
    measurements_A - Alice's measurements (Complex array)
    k_B - number of results per Bob's measurement (Integer)
    m_B - number of Bob measurements (Integer)

    Output:
    P - Problem created
    measurements_B - Bob's measurements created
    D - Value of Bell's inequality 
    solution - Result of the problem 
    '''

    k_A = int(measurements_A.shape[1])
    m_A = int(measurements_A.shape[0]/k_A)

    #Creating the problem
    P = pic.Problem()

    #Medição computacional
    KetBra_00 = np.array([[1,0],[0,0]])
    KetBra_11 = np.array([[0,0],[0,1]])

    ket_mais = (np.array([[1],[1]]))/np.sqrt(2)
    ket_menos = (np.array([[1],[-1]]))/np.sqrt(2)

    KetBra_mais = ket_mais@np.transpose(ket_mais)
    KetBra_menos = ket_menos@np.transpose(ket_menos)

    # M_B = np.zeros((2,2,2,2),dtype=complex)

    # for i in range(2):
    #     M_B[i] = measurement(2,2)

    # M_B = M_B.reshape(4,2,2)

    measurements_B = np.array([KetBra_00,KetBra_11,KetBra_mais,KetBra_menos])

    #Creating the optimization variables

    # measurements_B = [pic.HermitianVariable('Measurements_Bob[{}]'.format(i),(dimension,dimension)) for i in range(k_B*m_B)]

    # #Creating the constraints
    # P.add_list_of_constraints([measurements_B[i]>>0 for i in range(k_B*m_B)])

    # ident = [pic.sum([measurements_B[m_B*j+i] for i in range(k_B)]) for j in range(m_B)]

    # P.add_list_of_constraints([ident[j]==np.eye(dimension,dtype='complex') for j in range(m_B)]) 

    trace = [pic.trace((measurements_A[i]@measurements_B[j])*rho) for i in range(m_A*k_A) for j in range(m_B*k_B)]

    D = pic.sum([c[i]*trace[i] for i in range(c.shape[0])])

    #Setting the objective
    P.set_objective('max',D)

    #Finding the solution
    solution = P.solve()

    return P, measurements_B, D, solution

In [21]:
def Comp_measu():
    '''
    Generating the computational measurements 00, 11, + and -.
    '''

    #Medição computacional
    KetBra_00 = np.array([[1,0],[0,0]])
    KetBra_11 = np.array([[0,0],[0,1]])

    ket_mais = (np.array([[1],[1]]))/np.sqrt(2)
    ket_menos = (np.array([[1],[-1]]))/np.sqrt(2)

    KetBra_mais = ket_mais@np.transpose(ket_mais)
    KetBra_menos = ket_menos@np.transpose(ket_menos)

    return KetBra_00, KetBra_11, KetBra_mais, KetBra_menos

KetBra_00, KetBra_11, KetBra_mais, KetBra_menos = Comp_measu()

print(KetBra_00, KetBra_11, KetBra_mais, KetBra_menos)

[[1 0]
 [0 0]] [[0 0]
 [0 1]] [[0.5 0.5]
 [0.5 0.5]] [[ 0.5 -0.5]
 [-0.5  0.5]]


In [22]:
def max_Measurements(dimension,c,rho,measurements,k,m, side):

    '''
    Inputs:
    dimension - dimension of the measurements (Integer)
    c - Bell inequality constants (List of floats)
    rho - target state (Complex array)
    measurements- Other side measurements (Complex array)
    k - number of results per measurement for this side (Integer)
    m - number of measurements for this side (Integer)
    side - 'Alice' or 'Bob' (String)

    Output:
    P - Problem created
    measurements_new - Measurements created
    D - Value of Bell's inequality 
    solution - Result of the problem 
    '''

    k_o = int(measurements.shape[1])
    m_o = int(measurements.shape[0]/k_o)

    #Creating the problem
    P = pic.Problem()

    #Creating the optimization variables
    measurements_side = [pic.HermitianVariable('Measurements_'+side+'[{}]'.format(i),(dimension,dimension)) for i in range(k_B*m_B)]

    #Creating the constraints
    P.add_list_of_constraints([measurements_side[i]>>0 for i in range(k*m)])

    ident = [pic.sum([measurements_side[m*j+i] for i in range(k)]) for j in range(m)]

    P.add_list_of_constraints([ident[j]==np.eye(dimension,dtype='complex') for j in range(m)]) 

    if side=='Alice':
        trace = [pic.trace((measurements_side[i]@measurements[j])*rho) for i in range(m*k) for j in range(m_o*k_o)]
    elif side=='Bob':
        trace = [pic.trace((measurements[i]@measurements_side[j])*rho) for i in range(m_o*k_o) for j in range(m*k)]
    
    
    D = pic.sum([c[i]*trace[i] for i in range(c.shape[0])]).real

    #Setting the objective
    P.set_objective('max',D)

    #Finding the solution
    solution = P.solve()

    return P, np.array(measurements_side), float(D), solution


# KetBra_00, KetBra_11, KetBra_mais, KetBra_menos = Comp_measu()

measurements_A = np.array([KetBra_00,KetBra_11,KetBra_mais,KetBra_menos])
print(measurements_A)
# measurements_A = np.array([],dtype=complex)
# print(measurements_A)
# for i in range(2):
#     measurements_A = np.insert(measurements_A,proj_meas_random(2))
# print(measurements_A)
k_B = 2
m_B = 2
dim = 2

ket_01 = np.array([[0],[1],[0],[0]])
ket_10 = np.array([[0],[0],[1],[0]])
#Create |psi> = (|01>-|10>)/sqrt(2) and rho = |psi><psi|
psi = (ket_01-ket_10)/np.sqrt(2)
rho = psi@np.transpose(psi)

c = G_CHSH()

D_ant = 0
i=0
delta = 1

while delta > 10**(-7) and i<10^4:
    P, measurements_B, D, solution = max_Measurements(dim,c,rho,measurements_A,k_B,m_B,'Bob')
    P, measurements_A, D, solution = max_Measurements(dim,c,rho,measurements_B,k_B,m_B,'Alice')
    print(D)

    delta = np.abs(D_ant-D)
    D_ant = D
    i+=1


[[[ 1.   0. ]
  [ 0.   0. ]]

 [[ 0.   0. ]
  [ 0.   1. ]]

 [[ 0.5  0.5]
  [ 0.5  0.5]]

 [[ 0.5 -0.5]
  [-0.5  0.5]]]
2.0000000218522516
3.9999999997103868
3.9999999997219806


In [23]:
def SewSaw(dim,c,rho,k_A,m_A,k_B,m_B,rounds):

    results = np.zeros(rounds)

    for j in range(rounds):
        D_ant = 0
        i=0
        delta = 1
        
        measurements_A = np.zeros((k_A*m_A,dim,dim),dtype = complex)

        for i in range(m_A):

            random_meas = proj_meas_random(dim)

            for k in range(k_A):

                measurements_A[i*k_A+k] = random_meas[k]

        while delta > 10**(-7) and i<10^4:
            P, measurements_B, D, solution = max_Measurements(dim,c,rho,measurements_A,k_B,m_B,'Bob')
            P, measurements_A, D, solution = max_Measurements(dim,c,rho,measurements_B,k_A,m_A,'Alice')

            delta = np.abs(D_ant-D)
            D_ant = D
            i+=1
        results[j] = D
    
    print(results)

    return results.max()


k_B = 2
m_B = 2
k_A = 2
m_A = 2
dim = 2

ket_01 = np.array([[0],[1],[0],[0]])
ket_10 = np.array([[0],[0],[1],[0]])
#Create |psi> = (|01>-|10>)/sqrt(2) and rho = |psi><psi|
psi = (ket_01-ket_10)/np.sqrt(2)
rho = psi@np.transpose(psi)

c = G_CHSH()

D = SewSaw(dim,c,rho,k_A,m_A,k_B,m_B,2)

[4. 4.]


In [24]:
def LHS_threshold(rho_target,rho_sep):
    # Creating the measurements
    medicoes, eta = ut.measurements(3)
    m = int(medicoes.shape[0]/2)

    # Creating the deterministic strategies
    detp = fc.strategies_LHS(m,2)

    # Finding the threshold for locality

    P,solution,q = ut.SDP_LHS(m,2,rho_target,rho_sep,eta,detp,medicoes)

    return q

In [25]:
measurements_A = np.zeros((4,2,2),dtype = complex)
print(measurements_A)
print(proj_meas_random(2))
print('oi')
for i in range(2):
    print(measurements_A[i*(2):i*(2)+2-1].shape)
    measurements_A[i*(4):i*(4)+2-1] = proj_meas_random(2)
print(measurements_A)

[[[0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j]]]
[[[ 0.53125959-1.73356028e-18j  0.4983202 -2.64539740e-02j]
  [ 0.4983202 +2.64539740e-02j  0.46874041-6.14569927e-19j]]

 [[ 0.46874041-1.11407268e-17j -0.4983202 +2.64539740e-02j]
  [-0.4983202 -2.64539740e-02j  0.53125959+1.09357440e-17j]]]
oi
(1, 2, 2)


ValueError: could not broadcast input array from shape (2,2,2) into shape (1,2,2)

In [ ]:
c = G_CHSH()

#Create the base kets |00>, |01>, |10> and |11>
ket_01 = np.array([[0],[1],[0],[0]])
ket_10 = np.array([[0],[0],[1],[0]])

#Create |psi> = (|01>-|10>)/sqrt(2) and rho = |psi><psi|
psi = (ket_01-ket_10)/np.sqrt(2)
rho = psi@np.transpose(psi)

#Medição computacional
KetBra_00 = np.array([[1,0],[0,0]])
KetBra_11 = np.array([[0,0],[0,1]])

ket_mais = (np.array([[1],[1]]))/np.sqrt(2)
ket_menos = (np.array([[1],[-1]]))/np.sqrt(2)

KetBra_mais = ket_mais@np.transpose(ket_mais)
KetBra_menos = ket_menos@np.transpose(ket_menos)

# M_B = np.zeros((2,2,2,2),dtype=complex)

# for i in range(2):
#     M_B[i] = measurement(2,2)

# M_B = M_B.reshape(4,2,2)

M_B = np.array([KetBra_00,KetBra_11,KetBra_mais,KetBra_menos])
print(M_B)

P, M_A, D, solution = max_Measurements_Bob(2,c,rho,M_B,2,2)

print(M_A[0])
print(M_A[1])
print(M_A[2])
print(M_A[3])
print(D)
print(solution)
print(P)

P, M_B, D, solution = max_Measurements_Alice(2,c,rho,np.array(M_A),2,2)

print(M_B)
print(D)
print(solution)
print(P)

[[[ 1.   0. ]
  [ 0.   0. ]]

 [[ 0.   0. ]
  [ 0.   1. ]]

 [[ 0.5  0.5]
  [ 0.5  0.5]]

 [[ 0.5 -0.5]
  [-0.5  0.5]]]


ValueError: operands could not be broadcast together with shapes (2,2) (4,4) 

## Algoritmo Sew Saw

Agora sim, unindo as funções pra gerar o algoritmo SewSaw

In [ ]:
# Definindo a desigualdade de Bell: CHSH

c = G_CHSH()

# Criando o estado alvo
#Create the base kets |00>, |01>, |10> and |11>
ket_01 = np.array([[0],[1],[0],[0]])
ket_10 = np.array([[0],[0],[1],[0]])
#Create |psi> = (|01>-|10>)/sqrt(2) and rho = |psi><psi|
psi = (ket_01-ket_10)/np.sqrt(2)
rho = psi@np.transpose(psi)

# Definindo o número de medições
m_A = 2
m_B = 2

# Definindo o número de resultados por medição
k_A = 2
k_B = 2

# Definindo a dimensão da Alice e do Bob
dim_A = 2
dim_B = 2

# def Sewsaw(c, rho, m_A, m_B, k_A, k_B, dim_A, dim_B):

# Loop para fugir de máximos locais

results = np.zeros(10,dtype=complex)

for j in range(10):

    # Criando um set de medições iniciais para a Alice
    M_A = np.zeros((m_A,k_A,dim_A,dim_A),dtype=complex)

    for i in range(2):
        M_A[i] = measurement(dim_A,k_A)

    M_A = M_A.reshape(m_A*k_A,dim_A,dim_A)

    conv = False
    D_ant = 100

    i = 0

    while conv == False:

        # Fazemos a otimização das medições do Bob

        P, M_B, D, solution = max_Measurements_Bob(dim_B,c,rho,M_A,k_B,m_B)

        M_B = np.array(M_B)

        # Fazemos a otimização das medições da Alice

        P, M_A, D, solution = max_Measurements_Alice(dim_A,c,rho,M_B,k_A,m_A)
        
        M_A = np.array(M_A)

        # Vemos se houve convergência

        if abs(np.array(D) - D_ant) <= 10**(-7):
            conv = True
        else: 
            conv = False
            D_ant = np.array(D)

        i +=1
    
    results[j] = np.array(D)

    # print(D)
    # print(i)
print('Results',results.max())

tr([2×2]⊗Measurements_Bob[0]·[4×4])
∑(tr([2×2]⊗Measurements_Bob[0]·[4×4]), …, -tr([2×2]⊗Measurements_Bob[3]·[4×4]))
tr([2×2]⊗Measurements_Bob[0]·[4×4])
∑(tr([2×2]⊗Measurements_Bob[0]·[4×4]), …, -tr([2×2]⊗Measurements_Bob[3]·[4×4]))
tr([2×2]⊗Measurements_Bob[0]·[4×4])
∑(tr([2×2]⊗Measurements_Bob[0]·[4×4]), …, -tr([2×2]⊗Measurements_Bob[3]·[4×4]))
tr([2×2]⊗Measurements_Bob[0]·[4×4])
∑(tr([2×2]⊗Measurements_Bob[0]·[4×4]), …, -tr([2×2]⊗Measurements_Bob[3]·[4×4]))
tr([2×2]⊗Measurements_Bob[0]·[4×4])
∑(tr([2×2]⊗Measurements_Bob[0]·[4×4]), …, -tr([2×2]⊗Measurements_Bob[3]·[4×4]))
tr([2×2]⊗Measurements_Bob[0]·[4×4])
∑(tr([2×2]⊗Measurements_Bob[0]·[4×4]), …, -tr([2×2]⊗Measurements_Bob[3]·[4×4]))
tr([2×2]⊗Measurements_Bob[0]·[4×4])
∑(tr([2×2]⊗Measurements_Bob[0]·[4×4]), …, -tr([2×2]⊗Measurements_Bob[3]·[4×4]))
tr([2×2]⊗Measurements_Bob[0]·[4×4])
∑(tr([2×2]⊗Measurements_Bob[0]·[4×4]), …, -tr([2×2]⊗Measurements_Bob[3]·[4×4]))
tr([2×2]⊗Measurements_Bob[0]·[4×4])
∑(tr([2×2]⊗Measurements_Bob[0]·[4×4]